CashFlow

Adds currency and asset type filters

In [60]:
import pandas as pd
import os

#r'C:\Users\gusta\Documents\KTH\TriOptima\trioptima\'
#'/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/'
your_path = r'C:\Users\gusta\Documents\KTH\TriOptima\trioptima/'
folder_path = your_path + '6.Active Data/Train Model Data/'
# Get file in the folder
files = os.listdir(folder_path)
# MAC issue
files = [f for f in files if f != '.DS_Store']
data_file = files[0][:-4]

# Load the data
data = pd.read_csv(your_path + "2.Cleaned/" + data_file +'_Cleaned.csv')

# Filter trades based on conditions
filtered_data = data[
    (data['leg1NotionalCurrency'].isin(['EUR', 'USD', 'GBP'])) & 
    (data['leg1UnderlyingAssetOrContractType'].isin(['Fixed-Floating', 'OIS']))
]

print(filtered_data['leg2UnderlierID'].unique())
print(len(filtered_data['leg2UnderlierID'].unique()))





['USD-LIBOR-BBA' 'EUR-EURIBOR' 'USD-Federal Funds-H.15-OIS-COMPOUND'
 'EUR-EuroSTR-OIS Compound' 'GBP-SONIA-OIS Compound'
 'USD-SOFR-OIS Compound' 'USD-SOFR-1M']
7


Calculates discounted cashflows for Fixed-Floating and OIS

In [62]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta


df = pd.DataFrame(filtered_data)


# Convert date columns to datetime type
df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])
df['expirationDate'] = pd.to_datetime(df['expirationDate'])


def calculate_cashflows(row, period, freq):
    cashflows = []
    
    start_date = row['effectiveDate']
    end_date = row['expirationDate']
    # Calculate the payment frequency in days
    if row[period] == 'MNTH':
        delta = relativedelta(months=row[freq])
    elif row[period] == 'YEAR':
        delta = relativedelta(years=row[freq])
    
    elif row[period] == 'DAIL':
        delta = timedelta(days=row[freq])

    else:
        raise ValueError(f"Unknown frequency period: {row[period]}")
    
    while start_date < end_date:
        cashflows.append(start_date)
        start_date += delta

    return cashflows


df['cashflow_dates'] = df.apply(
    lambda row: 
        calculate_cashflows(row, 'leg1FixedRatePaymentFrequencyPeriod', 'leg1FixedRatePaymentFrequencyMultiplier') 
        if row['leg1UnderlyingAssetOrContractType'] == 'Fixed-Floating' 
        else (calculate_cashflows(row, 'leg2UnderlierTenorPeriod', 'leg2UnderlierTenorMultiplier') 
              if row['leg1UnderlyingAssetOrContractType'] == 'OIS' 
              else None), 
    axis=1)



# Define a generic discount rate curve
def discount_rate_curve(date):
    # Return a constant rate for simplicity, but in practice this can be any function of date
    return 0.03

# Calculate discounted cashflows
def calculate_discounted_cashflow(row, rate, notional):
    total_discounted_cashflow = 0
    
    for date in row['cashflow_dates']:
        cashflow = row[notional] * row[rate]
        # Calculate time difference in years for discounting
        time_difference = (date - datetime.now()).days / 365.0
        # Calculate the present value of the cashflow
        discounted_cashflow = cashflow / (1 + discount_rate_curve(date))**time_difference
        total_discounted_cashflow += discounted_cashflow
        
    return total_discounted_cashflow

df['discountedCashFlows'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'leg1FixedRate', 'leg1NotionalAmount'), axis=1)
df['discountedCashFlows'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'leg2FloatRate', 'leg2NotionalAmount'), axis=1)
df.drop(columns=['cashflow_dates'], inplace=True)
df.to_csv(your_path + 'cashflow.csv', index=False)


# This DataFrame now contains cashflow dates and their respective discounted cashflows for each trade
#print(df[['cashflow_dates', 'discounted_cashflows']])


C:\Users\gusta\AppData\Local\Temp\ipykernel_16960\1197828469.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])
C:\Users\gusta\AppData\Local\Temp\ipykernel_16960\1197828469.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['expirationDate'] = pd.to_datetime(df['expirationDate'])


KeyError: 'leg2FloatRate'

Risk